In [1]:
import pandas as pd

In [402]:
nl2bash = pd.read_json('nl2bash-data.json', orient='index')

In [403]:
nl2bash.cmd = nl2bash.cmd.apply(lambda x: x[2:] if x.startswith('$ ') else x)

In [404]:
set(nl2bash.cmd)

{'find /home -type d -name testdir',
 "sort -k1,1 -k2,2 -t';' --stable some_data",
 'basename /path/to/dir/filename.txt .txt',
 'find . -name libEGL* | xargs rm -f',
 'FOLDERS=$(find $PWD -type d | paste -d, -s)',
 "ifconfig eth0 | grep 'inet addr:' | cut -d: -f2 | awk '{print $1}'",
 "find . -name '*.js'",
 'chmod 555 /home/sshtunnel/',
 'comm -23 second-file-sorted.txt first-file-sorted.txt',
 'find / -name file1',
 "find .  -type f  -name '*.txt' -exec wc -l {} \\; | awk '{total += $1} END{print total}'",
 'find /home/username/public_html/modules -type f -exec chmod 640 {} +',
 'find . -name "*.php" -exec sed -i \'s/foo/bar/g\' {} \\;',
 "find ~ -type f -mtime 0 -iname '*.mp3'",
 "find . -name '*$VERSION*'",
 'find . -type f -perm 0777 -print',
 'od -c foo |head -2',
 'find . -maxdepth 1 -type f -exec rm -f {} \\;',
 'mkdir destdir',
 'find . -perm -600 -print',
 'find ! -path "dir1" ! -path "dir2" -iname "*.mp3"',
 'find /u/netinst -print | xargs chgrp staff',
 'rm -f *.bak *~',
 '

In [405]:
nl2bash.cmd.nunique(), len(nl2bash)

(8541, 10347)

In [37]:
import bashlex
from bashlex import 

In [324]:
def try_parse(x):
    try:
        return bashlex.parse(x)
    except:
        return None

def recurrent_command_extractor_generator(x):
#     print(f'parsing node {x}')
    if x.kind == 'command':
#         print(f'\t parsed as command containing {x.parts[0].word}')
        yield x.parts[0].word
    elif x.kind == 'commandsubstitution':
#         print(f'\t parsed as commandsubstitution containing {x.command}')
        yield from recurrent_command_extractor_generator(x.command)
    if hasattr(x, 'parts'):
#         print(f'\t gone further into parts')
        for part in x.parts:
            yield from recurrent_command_extractor_generator(part)
#     print('considered leaf')
    
def extract_commands(x):
    parsed = try_parse(x)
    if parsed is None:
        return []
    
    return [c for c in recurrent_command_extractor_generator(parsed[0])]

In [406]:
nl2bash['cmdset'] = nl2bash.cmd.apply(lambda x: set(extract_commands(x)))

In [407]:
selector = nl2bash.cmdset.apply(lambda x: len(x & set(possible_commands)) < len(set([i.replace('$', '') for i in x]) & set(possible_commands)))

In [418]:
nl2bash.loc[selector, nl2bash.columns=='cmd'] = nl2bash[selector].cmd.apply(lambda x: x[1:])

In [420]:
nl2bash['cmdset'] = nl2bash.cmd.apply(lambda x: set(extract_commands(x)))

In [385]:
', '.join(nl2bash.cmdset.explode().value_counts().head(70).index)

'find, xargs, grep, sort, awk, sed, echo, cut, cat, head, wc, tr, sudo, ls, tail, uniq, mkdir, rsync, split, read, dirname, which, readlink, date, pwd, diff, cd, tee, ssh, df, chown, ln, comm, set, mount, ifconfig, basename, od, md5sum, rev, shopt, hostname, history, mktemp, column, tar, yes, nl, ping, dig, mv, whoami, cp, join, seq, paste, egrep, tac, cpio, less, cal, who, du, fold, rm, tree, more, gzip, chmod, uname'

In [382]:
with open('all_commands.txt') as f:
    possible_commands = [s.strip() for s in f.readlines()]

In [383]:
len(set(nl2bash.cmdset.explode().values))

453

In [386]:
import numpy as np

In [387]:
import yaml

In [388]:
with open('cathegories.yml') as f:    
    cats = yaml.safe_load(f)

In [389]:
all_cathegorized_utils = []

for cath in cats:
    for util in cath['utils']:
        all_cathegorized_utils.append(util)

In [390]:
len(set(all_cathegorized_utils)), len(all_cathegorized_utils)

(117, 117)

In [391]:
set(all_cathegorized_utils) - (set(nl2bash.cmdset.explode().values) & set(possible_commands))

{'free', 'rename', 'tree', 'uptime'}

In [392]:
(set(nl2bash.cmdset.explode().values) & set(possible_commands)) - set(all_cathegorized_utils)

set()

In [393]:
back_dict = {}

for cath in cats:
    for util in cath['utils']:
        back_dict[util] = cath['name']

In [422]:
nl2bash['cmdcath'] = nl2bash.cmdset.apply(lambda x: set([back_dict[i] for i in x if i in back_dict.keys()]))

In [423]:
nl2bash.cmdcath.apply(len).value_counts()

1    7332
2    2678
3     291
0      39
4       7
Name: cmdcath, dtype: int64

In [432]:
nl2bash[nl2bash.cmdcath.apply(len) == 0].cmd.str.endswith('\\').sum()

13

In [436]:
nl2bash[nl2bash.cmd.str.endswith('\\')]

,invocation,cmd,cmdset,cmdcath
23,"Add a line number to every line in ""long-file""",nl -ba long-file \,{},{}
1052,Delete all files with inode number 804180,find -inum 804180 -exec rm {} \,{},{}
1066,Delete all files/directories with node number ...,find . -inum $inum -exec rm {} \,{},{}
1796,Find all .sh files in the current directory tr...,"find . -name ""*.sh"" -exec rm -rf '{}' \",{},{}
2791,Find all files/directories with inode number 1...,find -inum 16187430 -exec mv {} new-test-file-...,{},{}
2954,Find all regular files under current directory...,"find . -type f -exec sed -i ""s/1\.2\.3\.4/5.6....",{},{}
3107,Find and delete all core files in the user's h...,find ~/ -name 'core*' -exec rm {} \,{},{}
3953,List *.txt files residing in the current direc...,find . -name *.txt -exec ls {} ;\,{find},{FDOPS}
5798,Receive pattern to search for on the standard ...,cat f.html | grep -o \,{},{}
7435,Search the entire file hierarchy for all regul...,find / -user olduser -type f -exec chown new...,{},{}


In [427]:
bashlex.parse('find . ( -name a.out -o -name *.o ) -print')

ParsingError: unexpected token '(' (position 7)

In [396]:
nl2bash.loc[5641].invocation

'Prints path to folder that contains target of the symbolic link ../../../../etc/passwd.'

In [397]:
nl2bash.loc[5641].cmdset

{'$(dirname $(readlink -e ../../../../etc/passwd))', 'dirname', 'readlink'}

In [364]:
nl2bash.loc[5641].cmd

'$(dirname $(readlink -e ../../../../etc/passwd))'

In [398]:
selector = nl2bash.cmdset.apply(lambda x: len(x & set(possible_commands)) < len(set([i.replace('$', '') for i in x]) & set(possible_commands)))

In [399]:
nl2bash[selector]

,invocation,cmd,cmdset,cmdcath
3773,Get domain name from dig reverse lookup.,$dig -x 8.8.8.8 | grep PTR | grep -o google.*,"{grep, $dig}",{TXTPROC}
3847,Gets domain name from dig reverse lookup.,$dig -x 8.8.8.8 | grep PTR | grep -o google.*,"{grep, $dig}",{TXTPROC}
8076,change owner of the file file.sh to user root,$sudo chown root file.sh,{$sudo},{}


In [401]:
print(nl2bash[nl2bash.cmd.str.startswith('$')])

                                             invocation  \
1400  Execute all arguments to a shell script and wr...   
3668  Follows symbolic link $BASH_SOURCE, and prints...   
3773           Get domain name from dig reverse lookup.   
3847          Gets domain name from dig reverse lookup.   
4186  List content of 'myfile' in a subshell and ret...   
5580  Prints directory where the executing script ($...   
5641  Prints path to folder that contains target of ...   
5642  Prints path to the folder that contains target...   
5658  Prints real path of the folder containing $0 f...   
8076      change owner of the file file.sh to user root   

                                                   cmd  \
1400                                    $@ | tee $FILE   
3668            $(dirname $(readlink -f $BASH_SOURCE))   
3773    $dig -x 8.8.8.8 | grep  PTR | grep -o google.*   
3847    $dig -x 8.8.8.8 | grep  PTR | grep -o google.*   
4186                                     $(cat myfile)   
55

In [301]:
nl2bash[nl2bash.cmdcath.apply(len) == 0].loc[10054].cmd

'founddata=`find . -name "filename including space" -print0`'

In [211]:
', '.join(set(nl2bash.cmdset.explode().values) & set(possible_commands))

'basename, cd, od, more, shred, fgrep, info, sudo, crontab, ssh-keygen, awk, nl, yes, echo, tee, bash, diff, history, chmod, grep, chgrp, rsync, bunzip2, pushd, column, w, tac, apropos, paste, egrep, uniq, seq, ssh, file, clear, shopt, man, who, sh, md5sum, popd, sed, ping, whoami, vim, sleep, tr, mount, read, cat, ln, chown, df, rm, wc, fg, groups, fold, ifconfig, du, uname, pwd, kill, join, su, source, nohup, jobs, tar, touch, split, mkdir, dirname, pstree, watch, wget, rmdir, hostname, tail, gcc, date, tmux, head, mktemp, bzip2, bg, cpio, env, gzip, cut, scp, true, cp, readlink, set, zcat, less, ls, find, dig, top, mv, ps, curl, rev, gunzip, which, zless, bind, comm, sort, false, xargs'

In [187]:
set(nl2bash.cmdset.explode().values) & set(possible_commands)

{'apropos',
 'awk',
 'basename',
 'bash',
 'bg',
 'bind',
 'bunzip2',
 'bzip2',
 'cat',
 'cd',
 'chgrp',
 'chmod',
 'chown',
 'clear',
 'column',
 'comm',
 'cp',
 'cpio',
 'crontab',
 'curl',
 'cut',
 'date',
 'df',
 'diff',
 'dig',
 'dirname',
 'du',
 'echo',
 'egrep',
 'env',
 'false',
 'fg',
 'fgrep',
 'file',
 'find',
 'fold',
 'gcc',
 'grep',
 'groups',
 'gunzip',
 'gzip',
 'head',
 'history',
 'hostname',
 'ifconfig',
 'info',
 'jobs',
 'join',
 'kill',
 'less',
 'ln',
 'ls',
 'man',
 'md5sum',
 'mkdir',
 'mktemp',
 'more',
 'mount',
 'mv',
 'nl',
 'nohup',
 'od',
 'paste',
 'ping',
 'popd',
 'ps',
 'pstree',
 'pushd',
 'pwd',
 'read',
 'readlink',
 'rev',
 'rm',
 'rmdir',
 'rsync',
 'scp',
 'sed',
 'seq',
 'set',
 'sh',
 'shopt',
 'shred',
 'sleep',
 'sort',
 'source',
 'split',
 'ssh',
 'ssh-keygen',
 'su',
 'sudo',
 'tac',
 'tail',
 'tar',
 'tee',
 'tmux',
 'top',
 'touch',
 'tr',
 'true',
 'uname',
 'uniq',
 'vim',
 'w',
 'watch',
 'wc',
 'wget',
 'which',
 'who',
 'whoami',


In [176]:
nl2bash.cmdset.explode().value_counts().head(40)

find        6496
xargs       1176
grep         520
sort         400
awk          370
sed          315
echo         256
cut          220
head         193
cat          193
wc           183
sudo         169
tail         152
ls           132
mkdir        131
uniq         127
tr           126
rsync        102
split        101
read          96
diff          74
cd            71
ssh           70
tee           69
df            68
chown         68
ln            66
set           63
comm          63
mount         60
ifconfig      55
column        52
history       51
shopt         51
rev           50
tar           49
od            49
yes           48
nl            47
ping          45
Name: cmdset, dtype: int64

In [112]:
prsd[0].parts[0]

'ifconfig'

In [127]:
nl2bash.cmd[332]

'ls /empty_dir/ | xargs -n10 chown root'

In [78]:
escapes = ''.join([chr(char) for char in range(1, 32)])
t = 'nl -ba long-file \\'

In [79]:
t.translate(dict.fromkeys(escapes))

'nl -ba long-file \\'

In [69]:
def used_commands(cmd_node):

[CommandNode(parts=[WordNode(parts=[] pos=(0, 8) word='ifconfig'), WordNode(parts=[] pos=(9, 13) word='eth0')] pos=(0, 13)),
 PipeNode(pipe='|' pos=(14, 15)),
 CommandNode(parts=[WordNode(parts=[] pos=(16, 20) word='grep'), WordNode(parts=[] pos=(21, 33) word='inet addr:')] pos=(16, 33)),
 PipeNode(pipe='|' pos=(34, 35)),
 CommandNode(parts=[WordNode(parts=[] pos=(36, 39) word='cut'), WordNode(parts=[] pos=(40, 43) word='-d:'), WordNode(parts=[] pos=(44, 47) word='-f2')] pos=(36, 47)),
 PipeNode(pipe='|' pos=(48, 49)),
 CommandNode(parts=[WordNode(parts=[] pos=(50, 53) word='awk'), WordNode(parts=[] pos=(54, 66) word='{print $1}')] pos=(50, 66))]

In [33]:
len(set(nl2cm.cmd) & set(nl2bash.cmd))

8542

In [35]:
nl2bash

,invocation,cmd
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`
...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst
10345,view the manual page of find,man find
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`"""
